# San Francisco Working Families Credit

Analysis of [eligibility criteria](https://www.sfhsa.org/services/jobs-money/working-families-credit-wfc-program/eligibility-wfc) for the San Francisco Working Families Credit:

> * Have never received the WFC before
* Have earned less than $53,930 in 2017
* Have filed your tax return by April 17 deadline
* Claim the Earned Income Tax Credit (EITC) on your federal income tax return (see table below for more information)
* Claim at least one qualifying dependent child on your federal income tax return
* Live in San Francisco

The income limit of \$53,930 is redundant with the EITC claiming, as this is the maximum AGI to claim the EITC, [per IRS guidelines](https://www.irs.gov/credits-deductions/individuals/earned-income-tax-credit/eitc-income-limits-maximum-credit-amounts).

This notebook uses national CPS data to estimate the share of all EITC filers that have a qualifying dependent child.

While these criteria concern 2017 thresholds, this analysis uses 2018 data and tax logic, as any reforms would be for the 2018 tax year (early 2019).

*Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis  |  Date run: 2018-03-08*

## Setup

### Imports

In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.io import show, output_notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# On Python 3.6 use "import urllib.request as url_lib".
import urllib as url_lib

In [2]:
tc.__version__

'0.16.1'

### Configuration

In [3]:
sns.set_style('white')
DPI = 75
mpl.rc('savefig', dpi=DPI)
mpl.rcParams['figure.dpi']= DPI
mpl.rcParams['figure.figsize'] = 6.4, 4.8  # Default.

In [4]:
mpl.rcParams['font.sans-serif'] = 'Roboto'
mpl.rcParams['font.family'] = 'sans-serif'

# Set title text color to dark gray (https://material.io/color) not black.
TITLE_COLOR = '#212121'
mpl.rcParams['text.color'] = TITLE_COLOR

# Axis titles and tick marks are medium gray.
AXIS_COLOR = '#757575'
mpl.rcParams['axes.labelcolor'] = AXIS_COLOR
mpl.rcParams['xtick.color'] = AXIS_COLOR
mpl.rcParams['ytick.color'] = AXIS_COLOR

# Use Seaborn's default color palette.
# https://stackoverflow.com/q/48958426/1840471 for reproducibility.
sns.set_palette(sns.color_palette())

In [6]:
# Show one decimal in tables.
pd.set_option('precision', 2)

### Constants

In [5]:
WFC_INC_C = 53930

## Data

Generate data using Tax-Calculator.

In [7]:
recs = tc.Records.cps_constructor()

In [8]:
calc = tc.Calculator(records=recs, policy=tc.Policy())
calc.advance_to_year(2018)
calc.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [14]:
df = calc.dataframe([
    's006', 'aftertax_income', 'XTOT',
    'EIC',  # Children eligible for EITC.
    'eitc',  # EITC amount.
    'c00100',  # AGI, used for EITC threshold.
    # Include non-cash benefits to subtract for cash income.
    # Vet and other benefits have some cash components, but unclear how much.
    'snap_ben', 'mcaid_ben', 'mcare_ben', 'vet_ben', 'other_ben'])

Flags for WFC.

In [20]:
df['has_EIC'] = df['EIC'] > 0

Summed variables.

In [42]:
df['s006_m'] = df['s006'] / 1e6
df['XTOT_m'] = df['XTOT'] * df['s006_m']
df['EIC_m'] = df['EIC'] * df['s006_m']
df['eitc_m'] = df['eitc'] * df['s006_m']
df['afti_m'] = df['aftertax_income'] * df['s006_m']

Limit to EITC recipients and compute shares of tax units and people.

In [43]:
eitcr = df[df['eitc'] > 0].copy()  # Avoids warning.
eitcr['s006_share'] = 100 * eitcr['s006'] / eitcr['s006'].sum()
eitcr['XTOT_share'] = 100 * eitcr['XTOT_m'] / eitcr['XTOT_m'].sum()

## Current policy

## Reforms

### Removing dependent restriction

In [39]:
eitcr.groupby(['has_EIC'])['s006_share', 'XTOT_share'].sum()

,s006_share,XTOT_share
has_EIC,,
False,24.24,14.45
True,75.76,85.55


Average EITC amount per tax unit and per person.

In [44]:
by_has_eitc = eitcr.groupby(['has_EIC'])['s006_m', 'XTOT_m', 'eitc_m'].sum()
by_has_eitc['avg_eitc'] = by_has_eitc['eitc_m'] / by_has_eitc['s006_m']
by_has_eitc['avg_eitc_pp'] = by_has_eitc['eitc_m'] / by_has_eitc['XTOT_m']
by_has_eitc

,s006_m,XTOT_m,eitc_m,avg_eitc,avg_eitc_pp
has_EIC,,,,,
False,6.11,9.23,1828.49,299.21,198.07
True,19.10,54.66,57864.26,3029.58,1058.62


### Making a share of the federal EITC

## Census

In [53]:
from census import Census
from us import states

In [ ]:
c.acs5.get('NAME', )

In [56]:
c = Census('d6ae38c07a1b52d139c729945d7c88ea266806b1')
c.acs5.get(('NAME', '0500000US06075')),
#           {'for': 'state:{}'.format(states.MD.fips)})

TypeError: get() takes at least 3 arguments (2 given)

https://api.census.gov/data/2015/acs1/variables.html

## Appendix

Tax unit size distribution.

In [52]:
eitcr.groupby('XTOT')['s006_share'].sum()

XTOT
1.0     3.34e+01
2.0     2.31e+01
3.0     1.75e+01
4.0     1.47e+01
5.0     7.40e+00
6.0     2.55e+00
7.0     9.45e-01
8.0     2.83e-01
9.0     1.19e-01
10.0    3.33e-02
11.0    1.65e-02
12.0    1.86e-04
13.0    1.95e-03
14.0    5.86e-04
Name: s006_share, dtype: float64

How many get $5k+?

In [50]:
eitcr.loc[eitcr['eitc'] >= 5000, 's006_share'].sum()

12.395280131589635